# [MPRG] Recurrent Neural Networkによる電力予測


---
## 目的
Recurrent Neural Networkを使って電力予測を行う．ここで，今回はRecurrent Neural Networkの一種である，Long Short Term Memory（LSTM）を使用する．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではChainerを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


### データのダウンロード
プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍する．

In [1]:
!wget -q http://www.mprg.cs.chubu.ac.jp/tutorial/ML_Lecture/SOLAR/data.zip
!unzip -q data.zip
!ls
!ls -R ./data 

data  data.zip	sample_data
./data:
test  train

./data/test:
BEMS_RNN_test_data.npy	BEMS_RNN_test_labels.npy

./data/train:
BEMS_RNN_train_data.npy  BEMS_RNN_train_labels.npy


## モジュールのインポート
はじめに必要なモジュールをインポートする．


### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算をChainerで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．


In [ ]:
from time import time
from os import path
import numpy as np
import matplotlib.pyplot as plt
import chainer
from chainer import cuda, optimizers
from chainer import Chain, Variable
import chainer.functions as F
import chainer.links as L

# GPUの確認
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

## データセットの読み込みと確認
学習データを読み込みます．

読み込んだデータを変換します．
ここで，delayは何時刻先の電力値を教師信号にするかを決定するためのパラメータです．
delay=1と設定した場合，ネットワークへ入力したデータの1時刻先の電力が正解ラベルとなります．

データのサイズを確認します．
ネットワークへの入力データサイズは34となっており，出力の値は対応する電力値の1つとなっています．

In [4]:
# 読み込み
train_x = np.load(path.join('data', 'train', 'BEMS_RNN_train_data.npy'))
train_y = np.load(path.join('data', 'train', 'BEMS_RNN_train_labels.npy'))
test_x  = np.load(path.join('data', 'test', 'BEMS_RNN_test_data.npy'))
test_y = np.load(path.join('data', 'test', 'BEMS_RNN_test_labels.npy'))

# 数時刻先の電力が正解データになるように変換
delay = 1
train_x = np.asarray(train_x[ : -delay])
train_y = np.asarray(train_y[delay : ])
test_x = np.asarray(test_x[ : -delay])
test_y = np.asarray(test_y[delay : ])

# データのサイズ確認
train_y = train_y.reshape(len(train_y), 1)
test_y = test_y.reshape(len(test_y), 1)
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(49999, 34) (49999, 1)
(9999, 34) (9999, 1)


## ネットワークモデルの定義
再帰型ニューラルネットワークを定義します．
ここでは，LSTM層1層，全結合層1層から構成されるネットワークとします．

LSTM層はRecurrent Neural Networkの一種です．
LSTMへの入力サイズはNoneとし，データにより変更できるようにしておきます．

\__call\__関数では，定義した層を接続して処理するように記述します．
この時，全結合層から出力された値はシグモイド関数へと入力され，最終的な出力結果をえます．
その後，mean_squared_error関数へ出力値と正解データを入力することで，誤差を計算します．誤差とネットワークの出力値の両方をreturnによって返しています．

また，LSTMをはじめとするRecurrent Neural Networkでは，内部に過去の入力情報から計算した値を保持しています．
reset_state関数では，この内部情報の初期化を行うような処理を記述しています．


In [ ]:
class LSTM(chainer.Chain):
    def __init__(self):
        super(LSTM, self).__init__()
        with self.init_scope():
            self.l1=L.LSTM(None, 136)
            self.l2=L.Linear(136, 1)

    def __call__(self, x, t):
        h = F.sigmoid(self.l1(x))
        h = F.sigmoid(self.l2(h))

    self.loss = F.mean_squared_error(h, t)
        return self.loss, h

    def reset_state(self):
        self.l1.reset_state()

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．







In [ ]:
model = LSTM()
model.to_gpu()

optimizer = chainer.optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(model)

## 学習データの変換

上で読み込んだ学習・テストデータは電力の推移を表した１つの時系列データとなっている．
ここでは，この一つの時系列データから，短い時間間隔で区切ったデータを作成することで，学習データの作成を行う．

まず，time_windowで1サンプルの時間窓を決定する．今回は10時刻で1サンプルと設定した．
その後，指定した時間窓でサンプルを抽出し，convert_train_x, とconvert_train_yへと保存する．


In [ ]:
time_window = 10

convert_train_x = []
convert_train_y = []
for idx_frame in range(len(train_x) - time_window):
    partial_data = train_x[idx_frame:idx_frame + time_window]
    partial_label = train_y[idx_frame:idx_frame + time_window]

    convert_train_x.append(partial_data)
    convert_train_y.append(partial_label)

convert_train_x = np.asarray(convert_train_x)
convert_train_y = np.asarray(convert_train_y)

print(convert_train_x.shape, convert_train_y.shape)

(49989, 10, 34) (49989, 10, 1)


ここでは，GPUに対応した行列演算モジュールのcupyを呼び出しており，学習およびテストデータをcupyの形式に変換します．
cupyはnumpyと互換性があります．

In [ ]:
xp = cuda.cupy
convert_train_x = xp.array(convert_train_x, dtype=xp.float32)
convert_train_y = xp.array(convert_train_y, dtype=xp.float32)
test_x = xp.array(test_x, dtype=xp.float32)
test_y = xp.array(test_y, dtype=xp.float32)

## 学習

１回の誤差を算出するデータ数（ミニバッチサイズ）100，学習エポック数を10とします．
学習データサイズを取得し，１エポック内における更新回数を求めます．
学習データは毎エポックでランダムに利用するため，numpyのpermutationという関数を利用します．
各更新において，学習用データと教師データをそれぞれxとtとし，to_gpu関数でGPUに転送します．
学習モデルにxを与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベルtとの誤差をsoftmax coross entropy誤差関数で算出します．
また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
batch_size = 100
epoch_num = 10

train_data_num = convert_train_x.shape[0]
iter_one_epoch = int(convert_train_x.shape[0]/batch_size)

start = time()
for epoch in range(epoch_num):
    total_loss = 0

    perm = xp.random.permutation(train_data_num)
    for i in range(0, train_data_num, batch_size):
        x = Variable(cuda.to_gpu(convert_train_x[perm[i:i+batch_size]]))
        t = Variable(cuda.to_gpu(convert_train_y[perm[i:i+batch_size]]))
    
        accum_loss = 0.0
        model.reset_state()

        for idx_window in range(time_window):
            loss, y = model(x[idx_window], t[idx_window])
            accum_loss += loss
            total_loss += loss.data
    
        optimizer.target.cleargrads()
        accum_loss.backward()
        accum_loss.unchain_backward()
        accum_loss = 0
        optimizer.update()
  
    elapsed_time = time() - start
    print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch+1, total_loss/iter_one_epoch, elapsed_time))

epoch: 1, mean loss: 0.0018333939, elapsed_time :27.719924449920654
epoch: 2, mean loss: 0.0007399092, elapsed_time :55.40241289138794
epoch: 3, mean loss: 0.00043068835, elapsed_time :83.04099631309509
epoch: 4, mean loss: 0.00033185436, elapsed_time :110.62580442428589
epoch: 5, mean loss: 0.00026116523, elapsed_time :138.16764521598816
epoch: 6, mean loss: 0.00023392595, elapsed_time :165.5917730331421
epoch: 7, mean loss: 0.00021323386, elapsed_time :193.07578206062317
epoch: 8, mean loss: 0.00022182321, elapsed_time :220.6056478023529
epoch: 9, mean loss: 0.00019488708, elapsed_time :248.09467935562134
epoch: 10, mean loss: 0.00019687603, elapsed_time :275.646723985672


## テスト
学習したネットワークモデルを用いて評価（予測結果の可視化）を行います．
可視化にはmatplotlibを用います

In [ ]:
convert_test_x = test_x.reshape(1, 9999, 34)
convert_test_y = test_y.reshape(1, 9999, 1)
x_test = Variable(cuda.to_gpu(convert_test_x))
t_test = Variable(cuda.to_gpu(convert_test_y))

prediction_result = []
model.reset_state()

for idx_window in range(convert_test_x.shape[1]):
    loss, y = model(x_test[:, idx_window], t_test[:, idx_window])
    prediction_result.append(y.data)

prediction_result = np.array(prediction_result).flatten()

plt.figure()
plt.plot(test_y.tolist(), color='red', label='true')
plt.plot(prediction_result.tolist(), color='blue', label='pred')
plt.legend()
plt.show()

 ## 課題
1. 学習の更新回数を10epoch以上にした場合の結果を比較しましょう．

2. LSTM層を増やした際の性能を比較しましょう．

3. time_windowとミニバッチサイズを変えて実験しましょう．